**Clustering algorithms use unsupervised learning methods. Therefore the datasets must have distinctive
features. The purpose of this study is to cluster  the students according to the scores.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

There is no null values and there is no necessery any data cleaning process.

In [ ]:
data = pd.read_csv("../input/StudentsPerformance.csv")
data.head()

**Principal Component Analysis(PCA) Implementation**

I'll cluster students according to the scores they take from the exams. But before that I want to reduce the size of my 3-dimensional scores data to 2 dimensions.

In [ ]:
x = data.loc[:, "math score":].values

from sklearn.decomposition import PCA
pca = PCA(n_components = 2, whiten= True) #witten = normalize
pca.fit(x)

x_pca = pca.transform(x)
print("Accuracy of PCA : ", sum(pca.explained_variance_ratio_))

In [ ]:
df = pd.DataFrame(dict(p1 = x_pca[:, 0], p2 = x_pca[:, 1] ))
data = pd.concat([data,df], axis=1)

Let's take a look at the reduced grade values in 2 dimensions.

In [ ]:
import seaborn as sns
g = sns.pairplot(data.loc[:,["gender", "p1", "p2"]], hue="gender",plot_kws=dict(alpha=0.4))

As seen above, male and female students can be clustered according to their scores. I would like to show how accurate a clustering can be done if I divide the students into two clusters without gender data.

**K-Means Algorithm**

Using the K-means algorithm, lets make a clustering according to P1 and P2 values.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 2)
clusters = kmeans.fit_predict(df)

Algorithm has been implemented. Now it's time to visualize.

In [ ]:
import matplotlib.pyplot as plt

df["clusters"] = clusters
data["clusters"] = clusters

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(17, 5))

ax1.scatter(data.p1[data.gender == "female"], data.p2[data.gender == "female"], alpha=0.5)
ax1.scatter(data.p1[data.gender == "male"], data.p2[data.gender == "male"], alpha=0.5)
ax1.set_title('Accurate Data')

ax2.scatter(df.p1, df.p2, alpha=0.6, color="black")
ax2.set_title('Data Without Clustering')

ax3.scatter(df.p1[df.clusters == 0], df.p2[df.clusters == 0], alpha=0.5)
ax3.scatter(df.p1[df.clusters == 1], df.p2[df.clusters == 1], alpha=0.5)
ax3.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1], color="yellow", s=150)
ax3.set_title('Data With K-Means Clustering Algorithm')

plt.show()